In [1]:
# import libraries
import os
import json
import pandas as pd
# check current path
current_path = os.getcwd()
os.chdir(current_path)
print(current_path)

b:\Ernst\bi_direction Ita and Lad\llama and gpt


###  Convert the dataset into JSONL before performin fine-tuning

In [ ]:
# Convert CSV file into .jsonl file
# We assume that the CSV file contains two columns (Source and target sentences)
!python json_converter_tai.py \
  --dataset_dir ./dataset \
  --file_name eng2lad_dataset \
  --batch_sample 10 

Output written to ./dataset/ita2lad_dataset_tai.jsonl
Output written to ./dataset/ita2lad_dataset_tai.jsonl


## Create FT model of Llama using Together AI

#### Check the file format

In [ ]:
from together.utils import check_file
file_name = 'dataset/eng2lad_dataset_tai.jsonl'
report = check_file(file_name)
print(report)
assert report["is_check_passed"] == True

{'is_check_passed': True, 'message': 'Checks passed', 'found': True, 'file_size': 5948638, 'utf8': True, 'line_type': True, 'text_field': True, 'key_value': True, 'has_min_samples': True, 'num_samples': 7256, 'load_json': True, 'filetype': 'jsonl'}


###  Uploading & checking the dataset to Together AI

In [ ]:
from together import Together

# Please set your API Key from Together AI
client = Together(api_key='xxx')

# Upload formatted data and get back the file ID
response = client.files.upload(file=file_name)
fileId = response.model_dump()["id"]
# Verify that the file was uploaded successfully
file_metadata = client.files.retrieve(fileId)
print(file_metadata)

Uploading file ita2lad_dataset_tai.jsonl: 100%|██████████| 5.95M/5.95M [00:02<00:00, 2.04MB/s]


id='file-97c1a990-4b7e-40a7-9d64-068c3845bc8a' object=<ObjectType.File: 'file'> created_at=1733389630 type=None purpose=<FilePurpose.FineTune: 'fine-tune'> filename='ita2lad_dataset_tai.jsonl' bytes=5948638 line_count=0 processed=True FileType='jsonl'


In [ ]:
# Checking the uploaded file in Together AI
import requests

url = "https://api.together.xyz/v1/files"

headers = {
    "accept": "application/json",
    "authorization": "Bearer xxxx" # xxxx = api
}

response = requests.get(url, headers=headers)

print(response.text)

{"data":[{"filename":"ita2lad_dataset.jsonl","bytes":3593764,"created_at":1732122800,"id":"file-4491c6e8-4729-4436-9126-20c8d35c4376","purpose":"fine-tune","object":"file","LineCount":0,"Processed":true,"FileType":"jsonl"},{"filename":"ita2lad_dataset.jsonl","bytes":3593764,"created_at":1732141850,"id":"file-02aa8605-8317-47ce-8b58-c14b47e47cef","purpose":"fine-tune","object":"file","LineCount":0,"Processed":true,"FileType":"jsonl"},{"filename":"ita2lad_dataset.jsonl","bytes":3610493,"created_at":1732204528,"id":"file-d87701a0-9d78-401b-8773-fa82120b319f","purpose":"fine-tune","object":"file","LineCount":0,"Processed":true,"FileType":"jsonl"},{"filename":"ita2lad_dataset_tai.jsonl","bytes":5948638,"created_at":1733389630,"id":"file-97c1a990-4b7e-40a7-9d64-068c3845bc8a","purpose":"fine-tune","object":"file","LineCount":0,"Processed":true,"FileType":"jsonl"}],"object":"list"}


## Starting a Fine-tuning job

In [28]:
# Trigger fine-tuning job
resp = client.fine_tuning.create(
    suffix="mt_ita_lad",
    model="meta-llama/Meta-Llama-3.1-70B-Instruct-Reference",
    training_file=fileId,
    n_epochs=3,
    batch_size=8,
    learning_rate=1e-5,
    lora = True,
    lora_r = 32
    #wandb_api_key=os.environ.get("WANDB_API_KEY"),
)

### Create FT model Using Fine-tune DB

###  Convert the test data into JSONL for inferencing

In [ ]:
# Convert form CSV to formatted .jsonl file
!python json_converter.py \
  --dataset_dir ./dataset \
  --file_name ita2lad_dataset \
  --batch_sample 5 

# Perform Inference on the Fine-tuned model

In [99]:
!python ft_test.py \
  --model_name ft_llama_31_70b \
  --dataset_dir ./dataset \
  --test_data test_3_ita2lad \
  --target_lang italian \
  --batch_size 5 \
  --save_dir ./save_results

id='8ee4e15fafc6c2ca-VIE' object=<ObjectType.ChatCompletion: 'chat.completion'> created=1733578332 model='ulinnuha/Meta-Llama-3.1-70B-Instruct-Reference-mt_ita_lad-1afe2238-aad5d18c' choices=[ChatCompletionChoicesData(index=0, logprobs=None, seed=1437344216052657400, finish_reason=<FinishReason.EOS: 'eos'>, message=ChatCompletionMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content="Here are the translations from Ladin (Val Badia variant) to Italian:\n\n['Era per la sua grande morvëia appoggiato su una grande rete piena di pesc di ogni forma e grandezza che muovevano la coda e si torcevano come un gruppo di anime disperate.', 'Se ti va bene', 'A Pinocchio crescono le orecchie da maiale e poi diventa un maiale vero e proprio e si mette a grufolare.', 'Sono portato anche io a compatire, perché, come vedi, non ho più legna da rompere per preparare quel discorso.', 'Sul burattino si ripone ogni speranza, era proprio sul punto di lasciarsi cadere a terra e di darla vinta;']", tool_call

id='8ee510de0c82c306-VIE' object=<ObjectType.ChatCompletion: 'chat.completion'> created=1733580277 model='ulinnuha/Meta-Llama-3.1-70B-Instruct-Reference-mt_ita_lad-1afe2238-aad5d18c' choices=[ChatCompletionChoicesData(index=0, logprobs=None, seed=9190462390734302000, finish_reason=<FinishReason.EOS: 'eos'>, message=ChatCompletionMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content="Here are the translations from Ladin (Val Badia variant) to Italian:\n\n1. Le colp ê sté tan sterch che le pe s’â ficé ite tla porta de lëgn cina itamez.\nItalian: Il colpo è stato così forte che il piede si è ficcato nella porta di legno fino a metà.\n\n2. Ci ch’ai ne paiass, sc’al foss sëgn bele mesanöt!\nItalian: Quello che non paghiamo, se fosse bello a mezzanotte!\n\n3. Pornanch’al ê rové söla spona dl mer, ti â Pinocchio dé naota na gran odlada al mer;\nItalian: Poi è arrivato sulla riva del mare, e Pinocchio ha dato una gran botta al mare;\n\n4. Deach’al n’ê inurcaltan nia plü bun da jí inant po

## Get the prediction results

In [12]:
# Set parameter for performing Evaluations
test_data = 'test_3'# test_2 / test_3
llm_model = 'ft_llama_31_70b' # gpt/ mt5
batch_size = 5
path_f = 'ladin2italian'

In [3]:
# metrics
import evaluate
rouge_score = evaluate.load("rouge")
bleu_score = evaluate.load("bleu")
chrf_score = evaluate.load("chrf")
sacrebleu_score = evaluate.load("sacrebleu")

c:\Users\asus\miniconda3\envs\new_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
#function to calculate evaluations
def eval_metrics(predictions, references):
    # Ensure both predictions and references are lists of strings
    predictions = [str(pred) if pred is not None else "" for pred in predictions]
    references = [str(ref) if ref is not None else "" for ref in references]

    # Compute scores
    
    result = rouge_score.compute(predictions=predictions, references=references, rouge_types=["rouge1", "rouge2", "rougeL"])
    score = sacrebleu_score.compute(
            predictions=predictions,
            references=references
        )
    result["sacrebleu"] = score["score"]
    bleu = bleu_score.compute(predictions=predictions, references=references)
    result["bleu"] = bleu['bleu']
    chrf = chrf_score.compute(predictions=predictions, references=references) ##The higher the value, the better the translations
    chrf_plus = chrf_score.compute(predictions=predictions, references=references, word_order=2)  # chrF++
    result["chrf++"] = chrf_plus["score"]  #
    result["chrf"] = chrf["score"] #The higher the value, the better the translations
            
    #print("ROUGE:", result["rouge1"])
    #print("BLEU:", result["bleu"])
    #print("CHRF:", result["chrf"])
            
    return result

In [5]:
prefixes = [
    "Here are the Italian translations:",
    "Here are the Ladin (Val Badia variant) translations:",
    "Here are the Ladin (Val Badia variant) translations",
    "Here are the Italian translations: ",
    "Here are the translations of the 5 Italian texts to Ladin (Val Badia variant):",
    "Here are the translations from Italian to Ladin (Val Badia variant):",
    "Here are the translations: ",
    "Here are the translations from Italian to Ladin (Val Badia variant):",
    "Here are the translations of the 4 Italian texts to Ladin (Val Badia variant):",
    "Here are the translations from Ladin (Val Badia variant) to Italian:",
    "Here are the translations of the 5 Ladin (Val Badia variant) texts to Italian:",
    "Here are the translations in Italian:",
    "Here are the translations of the 5 Ladin (Val Badia variant) texts into Italian:",
    "Here are the translations of the 5 ('Ladin (Val Badia variant)',) texts to Italian:",
    "Here are the Ladin (Val Badia variant):",
    "Here are the Ladin (Val Badia variant) versions:",
    "Here are the Ladin (Val Badia variant) translations: ",
    "Here are the Laads (Val Badia variant) translations:",
    "Here are the translations of the 5 Italian texts into Ladin (Val Badia variant):",
    "Here are the translations of the given Italian texts to Ladin (Val Badia variant):"
    
    
          
]

In [6]:
# Compare the translation result
def get_json_files(llm_model, test_data):
    # Get JSON files for specific translation test data using a specific llm
    # Define the file prefix file name
    file_prefix = (f'translation_{llm_model}_{test_data}_ita2lad_size of_{batch_size}_batch_')

    # List all files in the directory that start with the specified prefix
    save_dir = 'save_results'
    matching_files = matching_files = [f for f in os.listdir(f'{save_dir}/{path_f}') if f.startswith(file_prefix)]  #current_path+'/save_results'
    # Count the number of matching files
    num_files = len(matching_files)
    print(f"Found {num_files} files.")

    scores = {'rouge1': [], 'rouge2': [], 'rougeL': [], 'bleu': [], 'chrf': [], 'sacrebleu': [], 'chrf++': []}
    all_scores = []
    batch_start = 0
    # Open real data
    ref_data = pd.read_csv(f'dataset/{test_data}_ita2lad.csv')
    for i in range(num_files):
         # Slicing for the current batch of data
        real_data = ref_data.iloc[batch_start:batch_start + batch_size]
        batch_start = (i + 1) * batch_size
        print(f"Processing batch {i+1}, starting at index {batch_start}")

        # Get the real data as a list
        # DOnt forget to set the target language
        ################################################
        real_data = real_data['italian'].tolist()      #
        ################################################

        # Open and read the JSON files of translation result
        file_loc=os.path.join(save_dir, path_f+f'/{file_prefix}{i}.json')#save_dir
        print("load the json file", file_loc)
        f = open(file_loc, encoding='utf8')
        data = json.load(f)
        # Get the target translation using llm API
        # if json data is in str, convert to dict
        if isinstance(data, str):
            data = json.loads(data)

        # Ensure 'choices' exists and contains data
        if "choices" in data and data["choices"]:
            translation_output = data["choices"][0].get("message", {}).get("content", "")
            #print(translation_output)
            if "choices" in data and data["choices"]:
                translation_output = data["choices"][0].get("message", {}).get("content", "")
                if translation_output.strip():  # Check if content is not empty
                    #if isinstance(translation_output, str):
                    # Clean and process translation output
                    translation_output = translation_output.strip('```json\n').strip('```')
                     
                    # Remove the additional response
                    for prefix in prefixes:
                        if translation_output.startswith(prefix):
                            translation_output = translation_output[len(prefix):].strip()
                        
                    #translation_output = json.loads(translation_output)  # Parse the JSON output
                    
                    # Convert the remaining text to a Python list
                    try:                     
                        translations_result = eval(translation_output)  # Caution: Use `eval` only if you're sure of the data source
                        # calculate the evaluation metric score
                        if len(translations_result) == len(real_data):
                            scores =eval_metrics(translations_result, real_data)
                        else:
                            print('the sentences do not match')
                             
                        scores =eval_metrics(translations_result, real_data)
                        all_scores.append(
                                {'rouge1': scores['rouge1'],
                                'rouge2':scores['rouge2'],
                                'rougeL': scores['rougeL'],
                                'bleu': scores['bleu'],
                                'chrf': scores['chrf'],
                                'sacrebleu': scores['sacrebleu'],
                                'chrf++': scores['chrf++']}
                                )

                    except Exception as e:
                        print(translation_output)
                        print(f"Error parsing the translations: {e}")
            else:
                print("No translations found.")
    return all_scores


In [13]:
# Get the evaluation scores    ’
translation_result=get_json_files(llm_model, test_data)

Found 313 files.
Processing batch 1, starting at index 5
load the json file save_results\ladin2italian/translation_ft_llama_31_70b_test_3_ita2lad_size of_5_batch_0.json
Processing batch 2, starting at index 10
load the json file save_results\ladin2italian/translation_ft_llama_31_70b_test_3_ita2lad_size of_5_batch_1.json
Processing batch 3, starting at index 15
load the json file save_results\ladin2italian/translation_ft_llama_31_70b_test_3_ita2lad_size of_5_batch_2.json
Processing batch 4, starting at index 20
load the json file save_results\ladin2italian/translation_ft_llama_31_70b_test_3_ita2lad_size of_5_batch_3.json
Processing batch 5, starting at index 25
load the json file save_results\ladin2italian/translation_ft_llama_31_70b_test_3_ita2lad_size of_5_batch_4.json
Processing batch 6, starting at index 30
load the json file save_results\ladin2italian/translation_ft_llama_31_70b_test_3_ita2lad_size of_5_batch_5.json
Processing batch 7, starting at index 35
load the json file save_r

In [14]:
# Convert the list of dictionaries to a DataFrame
fr = pd.DataFrame(translation_result)
print(len(translation_result))
# Calculate the mean for each column
mean_scores = fr.mean()
# Print the mean scoresb 
print(mean_scores)
llm_model, test_data, path_f

307
rouge1        0.383036
rouge2        0.175705
rougeL        0.366172
bleu          0.120030
chrf         37.935181
sacrebleu    13.017793
chrf++       36.117098
dtype: float64


('ft_llama_31_70b', 'test_3', 'ladin2italian')